In [ ]:
# Make the cell's full width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

import os
import sys
import quippy
import ase.io
import numpy             as     np
import matplotlib.pyplot as     plt

from tqdm                import tqdm

# Variables

Everything you need to modify should be in this cell. 

You just need to:
- Point to your training set
- 2b GAP
- root file save location
- Put your isolated energies in a dictionary indexed by atomic number
- The percentiles you want delta calculations for
- The number of SOAP environments
- The number of 2b bonds (both read from gap_fit log file)

The top three variables are for formating the plots, so they'll change baised on your use case

In [ ]:
fontsize          = 30
ulim              = 30        # Upper limit on volume per atom [Angs^3] (for plotting - 2b delta calculation)
E2b_Edft_lims     = (-5, 2.5) # x limits for the E_{2b} - E_{dft} plot (SOAP delta calculation)

dft_energy_key    = "dft_energy"
training_xyz      = "/storage/mssnkt_grp/msufgx/gits/SiC-framework/testing-dir/GAP/avon/MPI/G52-delta/in.training.xyz"
gap_2b            = "/storage/mssnkt_grp/msufgx/gits/SiC-framework/testing-dir/GAP/avon/MPI/G52-2b-only_delta0.8/gp.xml"
root              = "/storage/mssnkt_grp/msufgx/gits/SiC-framework/testing-dir/GAP/avon/MPI/G52-2b-only_delta0.8"

# Atomic number and zero energy as a dictionary
isolated_energies = {6 : -237.1146139071856, 14 : -615.4623832973588}

# Percentiles +/- to check delta values for
percentiles       = [1, 5, 10, 20]

#                   (# C SOAP + # Si SOAP) / 2
number_soap       = (690068 + 682190) / 2

#                   (# CC 2b + # SiSi 2b + # SiC 2b) / 3
number_2b         = (10537722 + 9490102 + 19008978) / 3

# Load the training dataset

In [ ]:
# Load Training set
sys.stdout.write("Loading training dataset... ")
sys.stdout.flush()
al = ase.io.read(training_xyz, index=":")
sys.stdout.write("Done!\n")
sys.stdout.flush()

# Calculate 2b Delta

In [ ]:
%matplotlib inline

binding_dft_energy_pa = np.zeros(len(al))
volume_pa             = np.zeros(len(al))
for i, a in enumerate(al):
    # Binding energy per atom
    #  (E_{DFT} - (ZeroEnergy_{species-1}*NumberOf_{species-1} + ZeroEnergy_{species-n}*NumberOf_{species-n})) / NumberOfAtoms
    #
    dft_energy           = a.info[dft_energy_key]
    total_binding_energy = 0
    for species in isolated_energies.keys():
        n_species = len([x for x in a.arrays["numbers"] if x == species])
        total_binding_energy += n_species * isolated_energies[species]
    
    binding_dft_energy_pa[i] = (dft_energy - total_binding_energy) / len(a)
    volume_pa[i]             = a.get_volume() / len(a)
    
#------------------------
# Figure
#------------------------
fig, axes = plt.subplots(1,2, figsize=(20,10), facecolor="w")

ax        = axes[0]
ax_zoomed = axes[1]

# Plot Data
x         = binding_dft_energy_pa
ax.scatter(x, volume_pa)
ax_zoomed.scatter(x, volume_pa)

ax_ymin, ax_ymax = ax.get_ylim()

deltas_2b = []
#------------------------
# 2b Deltas
#------------------------
print("2b delta = Range    * # SOAP descriptors / # 2b descriptors")
print("2b delta = Range    * {:>18.2f} / {:>.2f}".format(number_soap, number_2b))
print("2b delta = Range    * {:>.5f}".format(number_soap / number_2b))
print()

#------------------------
# Entire Range
#------------------------
dft_min   = np.amin(binding_dft_energy_pa)
dft_max   = np.amax(binding_dft_energy_pa)
dft_range = dft_max - dft_min
delta_2b  = dft_range * (number_soap / number_2b)

deltas_2b.append(delta_2b)

ax.vlines([dft_min, dft_max], ax_ymin, ax_ymax, color="k", label="0th & 100th Percentile")
ax_zoomed.vlines([dft_min, dft_max], -1, ulim+1, color="k")

print("DFT Binding Energy Spread (range) +/- {:>3d} %: {:>8.5f} eV/atom (min: {:>8.5f} eV/atom, max: {:>8.5f} eV/atom) | 2b delta = {:>8.5f} eV/bond".format(0, dft_range, dft_min, dft_max, delta_2b))

#------------------------
# Percentiles
#------------------------
for i, percentile_frac in enumerate(percentiles):
    _min, _max = np.percentile(x, [percentile_frac, 100 - percentile_frac])
    _range     = _max - _min
    delta_2b   = _range * (number_soap / number_2b)
    
    deltas_2b.append(delta_2b)
    
    ax.vlines([_min, _max], ax_ymin, ax_ymax, color="C{}".format(i+1), label="{}th & {}th Percentile".format(percentile_frac, 100 - percentile_frac))
    ax_zoomed.vlines([_min, _max], -1, ulim+1, color="C{}".format(i+1))
       
    print("DFT Binding Energy Spread (range) +/- {:>3d} %: {:>8.5f} eV/atom (min: {:>8.5f} eV/atom, max: {:>8.5f} eV/atom) | 2b delta = {:>8.5f} eV/bond".format(percentile_frac, _range, _min, _max, delta_2b))
print()



#------------------------
# Formatting
#------------------------
for _ax in axes:
    _ax.set_xlabel("$\mathrm{E_{DFT}^{Bind}}$ per atom [eV/atom]",  fontsize=fontsize)
    _ax.set_ylabel("Volume per atom [$\mathrm{\AA^3}$]", fontsize=fontsize)
    _ax.tick_params(axis="both", labelsize=fontsize)
    _ax.grid()

fig.suptitle("Data Spread (DFT) with Percentiles Shown", fontsize=fontsize+2)    
ax.set_title("Entire Spread", fontsize=fontsize+2)
ax_zoomed.set_title("Zoomed", fontsize=fontsize+2)
ax_zoomed.set_ylim(-0.2, ulim)
ax_zoomed.set_xlim(-10, -4)

ax.set_ylim(ax_ymin, ax_ymax)

handles, labels = ax.get_legend_handles_labels()

fig.legend(handles , labels, fontsize=fontsize, loc="lower center", bbox_to_anchor=(0.5, -0.4))

# Calculate 2b Energies

In [ ]:
if os.path.isfile(os.path.join(root, "2b_data.txt")):
    binding_2b_energy_pa = np.loadtxt(os.path.join(root, "2b_data.txt"))
else:
    calc = quippy.potential.Potential("IP GAP", param_filename=gap_2b)
    
    binding_2b_energy_pa = np.zeros(len(al))
    for i, a in enumerate(tqdm(al)):
        # Binding energy per atom
        #  (E_{DFT} - (ZeroEnergy_{species-1}*NumberOf_{species-1} + ZeroEnergy_{species-n}*NumberOf_{species-n})) / NumberOfAtoms
        #
        a.set_calculator(calc)
        two_body_energy      = a.get_potential_energy()
        total_binding_energy = 0
        for species in isolated_energies.keys():
            n_species = len([x for x in a.arrays["numbers"] if x == species])
            total_binding_energy += n_species * isolated_energies[species]

        binding_2b_energy_pa[i] = (two_body_energy - total_binding_energy) / len(a)

    np.savetxt(os.path.join(root, "2b_data.txt"), binding_2b_energy_pa)

# Calculate SOAP Delta

In [ ]:
%matplotlib inline
#------------------------
# Figure
#------------------------
fig, axes = plt.subplots(1,2, figsize=(20,10), facecolor="w")

ax        = axes[0]
ax_zoomed = axes[1]

# Plot Data
x         = binding_2b_energy_pa - binding_dft_energy_pa
ax.scatter(x, volume_pa)
ax_zoomed.scatter(x, volume_pa)

ax_ymin, ax_ymax = ax.get_ylim()

deltas_soap = []
#------------------------
# SOAP Deltas
#------------------------
print("SOAP delta = Range")

#------------------------
# Entire Range
#------------------------
two_body_min   = np.amin(binding_2b_energy_pa)
two_body_max   = np.amax(binding_2b_energy_pa)
two_body_range = two_body_max - two_body_min

deltas_soap.append(two_body_range)

ax.vlines([two_body_min, two_body_max], ax_ymin, ax_ymax, color="k", label="0th & 100th Percentile")
ax_zoomed.vlines([two_body_min, two_body_max], -1, ulim+1, color="k")

print("(2b Binding Energy - DFT Binding Energy) Spread +/- {:>3d} %: {:>8.5f} eV/atom (min: {:>8.5f} eV/atom, max: {:>8.5f} eV/atom) ".format(0, two_body_range, two_body_min, two_body_max))

#------------------------
# Percentiles
#------------------------
for i, percentile_frac in enumerate(percentiles):
    _min, _max = np.percentile(x, [percentile_frac, 100 - percentile_frac])
    _range     = _max - _min
    
    deltas_soap.append(_range)
    
    ax.vlines([_min, _max], ax_ymin, ax_ymax, color="C{}".format(i+1), label="{}th & {}th Percentile".format(percentile_frac, 100 - percentile_frac))
    ax_zoomed.vlines([_min, _max], -1, ulim+1, color="C{}".format(i+1))
    
    print("(2b Binding Energy - DFT Binding Energy) Spread +/- {:>3d} %: {:>8.5f} eV/atom (min: {:>8.5f} eV/atom, max: {:>8.5f} eV/atom) ".format(percentile_frac, _range, _min, _max))
print()

#------------------------
# Formatting
#------------------------
for _ax in axes:
    _ax.set_xlabel("$\mathrm{E^{Bind}_{2b} - E^{Bind}_{DFT}}$ per atom [eV/atom]",  fontsize=fontsize)
    _ax.set_ylabel("Volume per atom [$\mathrm{\AA^3}$]", fontsize=fontsize)
    _ax.tick_params(axis="both", labelsize=fontsize)
    _ax.grid()

fig.suptitle("Data Spread (for 2b - DFT) with Percentiles Shown", fontsize=fontsize+2)    
ax.set_title("Entire Spread", fontsize=fontsize+2)
ax_zoomed.set_title("Zoomed", fontsize=fontsize+2)
    
ax.plot([-20,20], [-20,20], c="grey", zorder=-10)


ax_zoomed.set_ylim(-0.2, ulim)
ax_zoomed.set_xlim(*E2b_Edft_lims)

ax.set_ylim(ax_ymin, ax_ymax)

handles, labels = ax.get_legend_handles_labels()

fig.legend(handles , labels, fontsize=fontsize, loc="lower center", bbox_to_anchor=(0.5, -0.4))

# Summary

In [ ]:
sum_percentiles = [0] + percentiles
for i, percentile_frac in enumerate(sum_percentiles):
    print("Percentile +/- {:>3d} % - 2b Delta: {:>8.5f}, SOAP Delta: {:>8.5f} | Ratio 2b/soap: {:>8.5f}".format(sum_percentiles[i], deltas_2b[i], deltas_soap[i], deltas_2b[i] / deltas_soap[i]))